In [1]:
import struct
import numpy as np

file_path = "gl05rl01pe08_org/boundary_GL05RL01.pe000002"  # Update with actual file path

# Mapping from datatype code to NumPy dtype
dtype_map = {
    0: "f4",  # real4 -> float32
    1: "f8",  # real8 -> float64
    2: "i4",  # int4 -> int32
    3: "i8"   # int8 -> int64
}

# Initialize an empty dictionary to store 9 NumPy arrays
data_arrays = {}

with open(file_path, "rb") as f:
    # Read the first 64 bytes (Header, Characters)
    f.read(64)
    
    # Read the next 256 bytes (Note, Characters)
    f.read(256)
    
    # Read the next 6 integers (4 bytes each) **before rgnid**
    _, _, _, _, _, num_of_rgn = struct.unpack(">6I", f.read(4 * 6))

    # Read the `rgnid` array correctly (num_of_rgn elements, each 4 bytes)
    f.read(4 * num_of_rgn)  # Skip rgnid array
    
    # Read `num_of_data` correctly after `rgnid`
    num_of_data = struct.unpack(">I", f.read(4))[0]  # Read 4-byte integer

    # Loop through all **9** data entries
    for entry_idx in range(9):  
        # Read metadata fields for this data entry
        varname = f.read(16).decode(errors="ignore").strip()
        f.read(64 + 16 + 16 + 256)  # Skip description, unit, layername, note

        # Read integer fields
        datasize, datatype, _, _ = struct.unpack(">Q3I", f.read(8 + 4 * 3))  # Q = int64, I = int32

        # Read time information
        f.read(8 * 2)  # Skip time_start, time_end

        # Ensure valid datatype before proceeding
        data_format = dtype_map.get(datatype, None)
        if not data_format:
            print(f"  ❌ Unknown datatype {datatype}, skipping data section.")
            f.seek(datasize, 1)  # Skip the data section
            continue

        # Compute the correct number of elements (datasize should match total bytes)
        element_size = np.dtype(data_format).itemsize  # Get byte size per element
        num_elements = datasize // element_size  # Number of elements

        # Read `1620` elements in Big-Endian format
        raw_data = np.frombuffer(f.read(datasize), dtype=np.dtype(">" + data_format))

        # Reshape into (324 points, 5 regions)
        reshaped_data = raw_data.reshape((324, 5))

        # Store in dictionary with the variable name
        data_arrays[varname] = reshaped_data

# Print available variable names
print("Extracted Variables:", list(data_arrays.keys()))

# Check shape of stored arrays
for key, value in data_arrays.items():
    print(f"{key}: shape {value.shape}")

# Output the dictionary containing all 9 NumPy arrays
data_arrays


Extracted Variables: ['grd_x_x\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'grd_x_y\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'grd_x_z\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_ix\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_jx\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_iy\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_jy\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_iz\x00\x00\x00\x00\x00\x00\x00', 'grd_xt_jz\x00\x00\x00\x00\x00\x00\x00']
grd_x_x         : shape (324, 5)
grd_x_y         : shape (324, 5)
grd_x_z         : shape (324, 5)
grd_xt_ix       : shape (324, 5)
grd_xt_jx       : shape (324, 5)
grd_xt_iy       : shape (324, 5)
grd_xt_jy       : shape (324, 5)
grd_xt_iz       : shape (324, 5)
grd_xt_jz       : shape (324, 5)


{'grd_x_x\x00\x00\x00\x00\x00\x00\x00\x00\x00': array([[-0.41662961, -0.44941392, -0.48159677, -0.51319546, -0.54419925],
        [-0.57457015, -0.60425306, -0.63318588, -0.66130571, -0.68855171],
        [-0.71486571, -0.74019184, -0.76447571, -0.78766363, -0.80970208],
        ...,
        [ 0.02018908,  0.0306444 ,  0.0411132 ,  0.05159223,  0.06206726],
        [ 0.07252042,  0.08293257,  0.09328418,  0.10355573,  0.11372795],
        [ 0.12378202,  0.13369964,  0.1434631 ,  0.15305532,  0.16245985]],
       shape=(324, 5), dtype='>f8'),
 'grd_x_y\x00\x00\x00\x00\x00\x00\x00\x00\x00': array([[ 0.34594806,  0.32651833,  0.30666376,  0.28654095,  0.26628263],
        [ 0.24599129,  0.22574368,  0.20559913,  0.18560695,  0.16581157],
        [ 0.14625526,  0.12697838,  0.10801683,  0.08939541,  0.07111628],
        ...,
        [-0.0621356 , -0.09431375, -0.12653341, -0.15878455, -0.19102337],
        [-0.2231949 , -0.25524021, -0.28709919, -0.31871175, -0.35001863],
        [-0.38096

In [3]:
data_arrays

{'grd_x_x\x00\x00\x00\x00\x00\x00\x00\x00\x00': array([[-0.41662961, -0.44941392, -0.48159677, -0.51319546, -0.54419925],
        [-0.57457015, -0.60425306, -0.63318588, -0.66130571, -0.68855171],
        [-0.71486571, -0.74019184, -0.76447571, -0.78766363, -0.80970208],
        ...,
        [ 0.02018908,  0.0306444 ,  0.0411132 ,  0.05159223,  0.06206726],
        [ 0.07252042,  0.08293257,  0.09328418,  0.10355573,  0.11372795],
        [ 0.12378202,  0.13369964,  0.1434631 ,  0.15305532,  0.16245985]],
       shape=(324, 5), dtype='>f8'),
 'grd_x_y\x00\x00\x00\x00\x00\x00\x00\x00\x00': array([[ 0.34594806,  0.32651833,  0.30666376,  0.28654095,  0.26628263],
        [ 0.24599129,  0.22574368,  0.20559913,  0.18560695,  0.16581157],
        [ 0.14625526,  0.12697838,  0.10801683,  0.08939541,  0.07111628],
        ...,
        [-0.0621356 , -0.09431375, -0.12653341, -0.15878455, -0.19102337],
        [-0.2231949 , -0.25524021, -0.28709919, -0.31871175, -0.35001863],
        [-0.38096